In [51]:
!git branch

  1-dataframe
  19-machine-tagger
  3-group-senses
* 4-semantic-provenance
  dev
  master
  oed-experiments


In [82]:
import pandas as pd
import json
from utils.dataset_download import *

In [15]:
lemma_id = 'machine_nn01'

In [79]:
# import API credentials
with open('oed_experiments/oed_credentials.json') as f:
    auth = json.load(f)

In [55]:
df = pd.read_pickle(f'./data/extended_{lemma_id}.pickle')

In [74]:
df.head(3)

,categories,daterange,definition,first_use,id,lemma,main_current_sense,meta,notes,oed_reference,oed_url,part_of_speech,provenance,provenance_type,quotation_ids,semantic_class_ids,semantic_class_last_id,transitivity,word_id
0,"{'topic': [], 'usage': [['rare']], 'region': []}","{'end': None, 'start': 1545, 'obsolete': False...","A material or immaterial structure, esp. the f...",J. Schäfer,machine_nn01-38473945,machine,False,"{'created': 1904, 'revised': True, 'updated': ...",[],"machine, n., sense I.1a",https://www.oed.com/view/Entry/111850#eid38473945,NN,"[[machine_nn01-38473945, seed, machine_nn01]]",seed,"[machine_nn01-38473950, machine_nn01-38473961,...","[[1, 111290, 118635, 119024, 120162, 120172], ...","[120172, 120173]",None,machine_nn01
1,"{'topic': [['Military', 'Weaponry']], 'usage':...","{'end': None, 'start': 1583, 'obsolete': False...",A military engine or siege-tower. Cf. war mach...,Brian Melbancke,machine_nn01-38474233,machine,False,"{'created': 1904, 'revised': True, 'updated': ...",[],"machine, n., sense II.3",https://www.oed.com/view/Entry/111850#eid38474233,NN,"[[machine_nn01-38474233, seed, machine_nn01]]",seed,"[machine_nn01-38474243, machine_nn01-38474252,...","[[153072, 160439, 163207, 163208, 163377, 1633...",[163378],None,machine_nn01
2,"{'topic': [], 'usage': [], 'region': []}","{'end': 1707, 'start': 1595, 'obsolete': True,...",spec. A scheme or plot. Obsolete.,Elizabeth I,machine_nn01-38474097,machine,False,"{'created': 1904, 'revised': True, 'updated': ...",[],"machine, n., sense I.1b",https://www.oed.com/view/Entry/111850#eid38474097,NN,"[[machine_nn01-38474097, seed, machine_nn01]]",seed,"[machine_nn01-38474102, machine_nn01-38474122,...","[[1, 84689, 87987, 87988, 87989, 88083, 88109,...",[88126],None,machine_nn01


In [77]:
len(set(df.id))

6985

In [67]:
def filter_by_year_range(dr: dict, start: int, end: int):
    """
    Arguments:
        dr (dict):
        start (int):
        end (int):
    
    Returns:
        bool
    """
    if dr.get('start',None) is None:
        dr_start = 0
    else:
        dr_start = dr['start']
    
    if dr.get('end',None) is None:
        dr_end = 2000
    else:
        dr_end = dr['end']
        
    if (dr_start <= start) and (dr_end >= end):
        return True
    
    return False

In [73]:
print(df.daterange.apply(filter_by_year_range, start=1760, end=1920)[:5])
print(df.iloc[0].daterange)
print(df.iloc[2].daterange)

0     True
1     True
2    False
3     True
4     True
Name: daterange, dtype: bool
{'end': None, 'start': 1545, 'obsolete': False, 'rangestring': '1545—'}
{'end': 1707, 'start': 1595, 'obsolete': True, 'rangestring': '1595—1707'}


In [57]:
set(df[df.id.isin(['machine_nn01-38475835','machine_nn01-38475923'])].definition)

{'A conceptual, abstract, or theoretical mechanism or device; spec. a model or a mathematical abstraction of an existing or hypothetical computer. Cf. Turing machine n.',
 'Mechanics. Anything that transmits force or directs its application.'}

In [40]:
def select_senses_by_provenance(sub_df: pd.DataFrame, sense_ids: set, relations: list):
    """
    Arguments:
        sub_df (pd.DataFrame):
        sense_ids (set):
        relations (list):
        
    Returns:
        a tuple contain as list with position indices and a list with senses
    """
    indices, senses = set(),set()
    for i, row in sub_df.iterrows():
        for oed_id, relation, prov_id in row.provenance:
            if (prov_id in sense_ids) and (relation in relations):
                senses.add(oed_id); indices.add(i)
                
    return list(indices), list(senses)

In [71]:
def obtain_senses(df, sense_ids:set, relations:list, start:int, end:int):
    """
    Arguments:
        df (pd.DataFrame):
        senses_ids (set):
        relations (list):
        start (int):
        end (int):
    Returns:
        set with senses
    """
    print(df.shape)
    df = df[df.daterange.apply(filter_by_year_range, start=start, end=end)]
    print(df.shape)
    seeds = df[df['provenance_type'] == "seed"].reset_index(inplace=False)
    synonyms = df[df['provenance_type'] == "synonym"].reset_index(inplace=False)
    branches = df[df['provenance_type'] == "branch"].reset_index(inplace=False)
    
    #print(seeds.shape[0],synonyms.shape[0],branches.shape[0])
    if "seed" in relations:
        seeds_selected = set(seeds[seeds.id.isin(sense_ids)].id)

    if "synonym" in relations:
        syn_sel_indices, synonyms_selected = select_senses_by_provenance(synonyms,sense_ids,relations)
    
    
    select_synonyms_semantic_class_id = synonyms[synonyms.id.isin(synonyms_selected)].semantic_class_last_id
    semantic_class_ids = set().union(*map(set,select_synonyms_semantic_class_id))
    
    branch_sel_indices, branches_selected = select_senses_by_provenance(branches,semantic_class_ids,relations)
    
    senses = set(branches.iloc[branch_sel_indices].id
               ).union(set(synonyms.iloc[syn_sel_indices].id)
                        ).union(set(seeds_selected))
    return senses
    
    
senses = obtain_senses(df,
                       {'machine_nn01-38475835','machine_nn01-38475923',},
                       ['seed','synonym','descendant'],
                       start=1760, 
                       end=1920
                      )

(8383, 19)
(2702, 19)


In [72]:
senses

{'accumulator_nn01-34592667',
 'lever_nn01-39377191',
 'machine_nn01-38475923',
 'mover_nn01-35820685',
 'pairing_nn02-32485248',
 'power_nn01-28687898',
 'primummobile_nn01-28369573',
 'screw_nn01-24083320',
 'wedge_nn01-14811588'}

In [84]:
def obtain_quotations(df: pd.DataFrame,
                      df_quotations:  pd.DataFrame,
                      senses):
    """
    Arguments:
        df (pd.DataFrame):
        df_quotations: pd.DataFrame
        senses (set):
        
    Returns:
        set with senses
    """
    pass